# Final Project Foundamentals of Data Science
### Solar Eclipse phases classification
---------
-   **Titolo del progetto:** Classificazione delle fasi dell'eclissi solare 
-   **Autori:** Luca Moresca, Nicholas Suozzi, Valerio Santini
-   **Descrizione del problema:** Fornire una breve panoramica del problema della classificazione delle fasi dell'eclissi solare. Evidenziare l'importanza dello studio delle eclissi solari per la comprensione dell'interazione tra Sole e Terra.
-   **Obiettivo del progetto:** Definire chiaramente l'obiettivo del progetto, ovvero sviluppare un modello in grado di classificare accuratamente le diverse fasi di un'eclissi solare da immagini.


### Abstract


In [1]:
if True:
    %pip install -qqq numpy scipy matplotlib pandas scikit-learn seaborn tqdm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%matplotlib inline
import numpy as np # imports a fast numerical programming library
import matplotlib.pyplot as plt # sets up plotting under plt
import matplotlib.image as mpimg
import pandas as pd # lets us handle data as dataframes
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import os
from pathlib import Path
%load_ext autoreload
%autoreload 2

# sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

## 1. Analisi del dataset

-   **Descrizione del dataset:** Descrivere il dataset di immagini di eclissi solare fornito in "ProgettoFDS.pdf", includendo il numero di immagini, le dimensioni delle immagini e le diverse classi di eclissi. [2, 3]
-   Elencare le classi di eclissi con le relative etichette numeriche: [2]
-   TotalSolarEclipse: 0
-   0to25percentPartialEclipse: 1
-   26to55percentPartialEclipse: 2
-   56to95percentPartialEclipse: 3
-   Darks: 4
-   DiamondRing_BaileysBeads_SolarEclipse: 5
-   Flats: 6
-   NotASolarEclipse: 7
-   **Visualizzazione del dataset:** Includere alcuni esempi di immagini dal dataset, mostrando le diverse fasi dell'eclissi.

The “Great American Eclipse” in 2017 offered a unique opportunity for US citizens across the entire continent to experience the awe-inspiring changes that occur during rare total solar eclipses (TSEs). The goal of the 2017 Eclipse Megamovie was to obtain enough pictures of the Great American Total Solar Eclipse to create a movie of the solar corona in order to better study it. Over 2,000 volunteers submitted 50,000 images and a movie was made only a few hours after the Moon's shadow left the U.S. A few months later another, better aligned and more closely edited, movie was released. The Eclipse Megamovie team of scientists processed images to create High Dynamic Range (HDR) images to capture the structures within the high dynamic range of the solar corona from the Sun's photosphere out ~3 solar radii.
https://eclipsemegamovie.org

Eclipse megamovie è un progetto che viene portato avanti da un gruppo di ricercatori in collaborazione con la NASA. Il punto centrale di tale progetto è quello di realizzare un video in grado di facilitare la ricerca su un evento raro come l'eclissi. Per realizzare questo progetto è stato chiesto ai cittadini americani di inviare fotografie scattate al sole durante l'eclissi. Il dataset contiene quindi immagini semi-professionali e amatoriali, scattate in condizioni non sempre ottimali. 

Eclipse megamovie is a project being carried out by a group of researchers in collaboration with NASA. The focus of this project is to make a video that can facilitate research into a rare event such as the eclipse that occurred in the United States in 2017. To realise this project, American citizens were asked to send in photographs taken during the eclipse. The dataset therefore contains semi-professional and amateur images, taken under conditions that are not always optimal.

In [ ]:
# Carica le immagini (sostituisci con i tuoi file)

desktop_path = Path.home() / "Desktop"
dataset_path = desktop_path / "eclupse"

img1 = mpimg.imread('/Users/valeriosantini/Desktop/eclipse-megamovie/train/00a0f91f0e7ece88ebaeab2a2050cdc0d72c5ea0b56c601dfaf800077df44c51.jpg')
img2 = mpimg.imread('/Users/valeriosantini/Desktop/eclipse-megamovie/train/00a4af4cc3f2a667c80331051a733e8dae2477d682d7dcef3e895f6f923dd86a.jpg')
img3 = mpimg.imread('/Users/valeriosantini/Desktop/eclipse-megamovie/train/00c11b16fd6426a88039c8b27e2511e2e770b65e0eb9daab720d3e6ca996bc06.jpg')
img4 = mpimg.imread('/Users/valeriosantini/Desktop/eclipse-megamovie/train/001e6704a60798684f1656713b593e6a56de81474e7173a31936589cd62f13e8.jpg')

# Imposta una figura con sottotrame affiancate
fig, axes = plt.subplots(1, 4, figsize=(16, 4))  # 1 riga, 4 colonne, dimensioni regolate
fig.tight_layout(pad=3.0)  # Regola la spaziatura tra le immagini

# Mostra ciascuna immagine in una sottotrama
axes[0].imshow(img1)
axes[0].set_title("Image 1")
axes[0].axis('off')  # Nasconde gli assi

axes[1].imshow(img2)
axes[1].set_title("Image 2")
axes[1].axis('off')

axes[2].imshow(img3)
axes[2].set_title("Image 3")
axes[2].axis('off')

axes[3].imshow(img4)
axes[3].set_title("Image 4")
axes[3].axis('off')

# Mostra la griglia di immagini
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/valeriosantini/Documents/GitHub/Corasoft-Unity/FDS/FInal Project/eclipse_megamovie/train/00a0f91f0e7ece88ebaeab2a2050cdc0d72c5ea0b56c601dfaf800077df44c51.jpg'

## 2. Preprocessing delle immagini

-   **Spiegazioni:** Descrivere le tecniche di preprocessing delle immagini utilizzate, spiegando la logica alla base di ogni scelta.
-   Ad esempio, se si ridimensionano le immagini, spiegare perché è necessario farlo e quale dimensione si è scelto di utilizzare.
-   Se si convertono le immagini in scala di grigi, giustificare questa scelta.
-   Se si normalizzano i valori dei pixel, spiegare il metodo di normalizzazione utilizzato e perché è importante.
-   **Codice:** Fornire il codice Python utilizzato per il preprocessing delle immagini.
-   **Visualizzazioni:** Mostrare alcuni esempi di immagini pre-elaborate, confrontandole con le immagini originali.


### Suggerimenti per il Preprocessing delle Immagini per la Classificazione delle Fasi di Eclissi Solare

Ecco alcuni suggerimenti per descrivere le tecniche di preprocessing delle immagini che potresti utilizzare per il tuo progetto di classificazione delle fasi di eclissi solare, insieme alla logica alla base di ogni scelta:

*   **Conversione in scala di grigi:** La conversione delle immagini in scala di grigi è una scelta comune nel preprocessing delle immagini, soprattutto quando il colore non è una caratteristica discriminante per il compito in questione. Nel caso della classificazione delle fasi di eclissi solare, le informazioni di colore potrebbero non essere essenziali per distinguere le diverse fasi. La conversione in scala di grigi riduce la complessità dei dati e la quantità di informazioni da elaborare, il che può accelerare i tempi di addestramento del modello e potenzialmente migliorare le sue prestazioni.
*   **Riduzione del rumore:** Le immagini possono contenere rumore che può influenzare negativamente le prestazioni del modello. La riduzione del rumore può essere realizzata utilizzando filtri come il filtro gaussiano.  Applicare un filtro gaussiano prima di altre operazioni di preprocessing può aiutare a migliorare la qualità delle immagini e a rendere più evidenti le caratteristiche importanti per la classificazione.
*   **Downscaling:**  Se le immagini originali hanno una risoluzione molto alta, il downscaling può essere utile per ridurre la quantità di dati da elaborare e accelerare i tempi di addestramento. La dimensione da utilizzare dipenderà dalla risoluzione originale delle immagini e dalla complessità del modello. È importante scegliere una dimensione che conservi le caratteristiche essenziali per la classificazione, evitando al contempo di perdere troppe informazioni.  Assicurati di applicare un filtro gaussiano prima del downscaling per evitare l'aliasing. 
*   **Normalizzazione:** La normalizzazione dei valori dei pixel è un'altra tecnica comune nel preprocessing delle immagini.  Può essere eseguita sottraendo la media e dividendo per la deviazione standard dei valori dei pixel. Questo processo porta tutti i valori dei pixel in un intervallo simile, il che può aiutare a migliorare le prestazioni del modello, soprattutto quando si utilizzano algoritmi di apprendimento automatico sensibili alla scala dei dati.

**Tecniche utili dagli homework:**

*   **Dal Homework 1:** Tutte le tecniche elencate (riduzione del rumore, downscaling, normalizzazione e conversione in scala di grigi) possono essere utili per il tuo progetto.
*   **Dal Homework 2:** La codifica one-hot sarà necessaria per le etichette delle fasi dell'eclissi. La gestione dei valori mancanti potrebbe essere utile se il tuo set di dati contiene immagini incomplete o danneggiate. L'espansione polinomiale e lo scaling potrebbero non essere necessari per il tuo progetto, in quanto sono tecniche più comunemente utilizzate per dati numerici non immagine.

### Tecniche Essenziali di Preprocessing per Immagini di Eclissi Solari Variabili

Dato che si ha a che fare con una vasta gamma di immagini scattate da amatori, che presentano diverse proporzioni, dimensioni del sole e posizioni del sole all'interno dell'immagine, è fondamentale applicare tecniche di preprocessing essenziali per preparare i dati per l'addestramento del modello di classificazione delle fasi dell'eclissi. 

Oltre alle tecniche di base già discusse (conversione in scala di grigi, riduzione del rumore, downscaling e normalizzazione), ecco alcune altre tecniche molto essenziali da considerare:

*   **Ritaglio e ridimensionamento:** Le immagini potrebbero avere bordi o aree irrilevanti che non contribuiscono alla classificazione delle fasi dell'eclissi. Il ritaglio può essere utilizzato per rimuovere queste aree e focalizzare l'attenzione sulla regione di interesse, ovvero il sole e la corona solare. Dopo il ritaglio, tutte le immagini dovrebbero essere ridimensionate a una dimensione uniforme per garantire la compatibilità con il modello. Poiché le proporzioni delle immagini originali possono variare, è possibile utilizzare un ridimensionamento con padding per mantenere le proporzioni originali, aggiungendo bordi neri attorno all'immagine ritagliata per raggiungere la dimensione desiderata. 
*   **Centratura del Sole:** La posizione del sole all'interno dell'immagine può variare notevolmente. Per migliorare le prestazioni del modello, è utile centrare il sole in tutte le immagini. Ciò può essere ottenuto tramite algoritmi di rilevamento degli oggetti o, in casi più semplici, analizzando l'istogramma dell'immagine per identificare la regione più luminosa (presumibilmente il sole) e spostarla al centro dell'immagine.
*   **Aumento dei dati:** Data la variabilità delle immagini, l'aumento dei dati può essere cruciale per migliorare la robustezza e la generalizzabilità del modello. Le tecniche di aumento includono:
    *   **Rotazione:** Ruotare le immagini di angoli casuali per aumentare la varietà di orientamenti del sole.
    *   **Ribaltamento:** Ribaltare le immagini orizzontalmente o verticalmente per simulare diverse prospettive.
    *   **Luminosità/Contrasto:** Variare la luminosità e il contrasto delle immagini per simulare diverse condizioni di illuminazione.

L'applicazione di queste tecniche di preprocessing contribuirà a creare un set di dati più uniforme e coerente, migliorando le prestazioni del modello di classificazione delle fasi dell'eclissi solare.



## 3. Estrazione delle feature

-   **Istogrammi:Spiegazioni:** Spiegare come gli istogrammi possono essere utilizzati per estrarre feature dalle immagini di eclissi solare. [1]
-   Descrivere come la distribuzione dei pixel nell'istogramma può variare a seconda della fase dell'eclissi.
-   **Codice:** Fornire il codice Python utilizzato per calcolare gli istogrammi.
-   **Visualizzazioni:** Mostrare alcuni esempi di istogrammi per le diverse fasi dell'eclissi.
-   **Altre tecniche (opzionale):** Se si utilizzano altre tecniche di estrazione delle feature, includere una sezione simile a quella degli istogrammi, spiegando e visualizzando i risultati.

## 4. Scelta e implementazione del modello

-   **Modello scelto:** Indicare chiaramente il modello di classificazione scelto per il progetto (ad esempio, Regressione Logistica, Regressione Softmax, CNN). [4-6]
-   Giustificare la scelta del modello in base alle caratteristiche del problema e del dataset.
-   Fornire una breve descrizione teorica del modello scelto, evidenziandone i principi di funzionamento.
-   **Codice:** Fornire il codice Python utilizzato per implementare il modello.
-   Utilizzare le competenze acquisite durante la realizzazione di "HomeWork.pdf" per implementare il modello in modo efficiente. [7]
-   Commentare il codice in modo chiaro per renderlo comprensibile.
-   **Training del modello:Spiegazioni:** Descrivere la procedura di training del modello, specificando la funzione di loss, l'algoritmo di ottimizzazione e gli iperparametri utilizzati. [8-20]
-   Spiegare come si è divisa il dataset in set di training e di test.
-   **Codice:** Fornire il codice Python utilizzato per il training del modello.
-   **Visualizzazioni:** Mostrare l'andamento della funzione di loss durante il training. [21-27]
-   Plottare anche l'accuratezza sul set di training e sul set di test durante il training.


## 5. Valutazione del modello

-   **Spiegazioni:** Descrivere le metriche utilizzate per valutare le prestazioni del modello. [13, 28-32]
-   Oltre all'accuratezza, considerare altre metriche come precisione, recall e F1-score, soprattutto se si ha uno sbilanciamento di classe.
-   **Codice:** Fornire il codice Python utilizzato per calcolare le metriche di valutazione.
-   **Risultati:** Presentare i risultati ottenuti dal modello sul set di test.
-   Utilizzare tabelle e grafici per visualizzare i risultati in modo chiaro ed efficace.
-   **Discussione:** Analizzare i risultati ottenuti, commentando le prestazioni del modello e le eventuali limitazioni.

## 6. Video delle transizioni dell'eclissi (opzionale) 

-   **Spiegazioni:** Descrivere il processo di creazione del video, includendo il software utilizzato e le scelte creative fatte.
-   Spiegare come si sono selezionate le immagini per il video e come si è creata la sequenza di transizione.
-   **Video:** Includere il video nel notebook Jupyter.

## 7. Conclusioni

-   **Riepilogo:** Riassumere brevemente il lavoro svolto nel progetto.
-   **Risultati principali:** Evidenziare i risultati più importanti ottenuti.
-   **Sviluppi futuri:** Suggerire eventuali miglioramenti o sviluppi futuri del progetto.